In [ ]:
###===================WARNING===================###
''' 
Þegar þessi hluti er keyrður þá mun hann reyna að 
pip intsall-a numpy, prettytable og matplotlib.

Kóðin mun spyrja um leyfi y/n fyrir hvern package
ef hann er ekki nú þegar til staðar.

Ef þeir eru nú þegar í tölvunni mun kóðin skila 
"{package name} is already installed."
'''
###=============================================###

import subprocess
import sys

required_packages = [
    "numpy"
]

def install_if_missing(package):
    try:
        __import__(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} is missing.")
        choice = input(f"Do you want to install '{package}'? (y/n): ").strip().lower()

        if choice == "y":
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        else:
            print(f"Skipped installing {package}.")

for pkg in required_packages:
    install_if_missing(pkg)

In [ ]:
def poisuilles(x,G,p1,p0):

    poisuilles_gildi = [
    (G,   p1,  x[0]),       # q_1A
    (G,   x[0], x[1]),      # q_AB
    (G,   x[0], x[2]),      # q_AC
    (G,   x[1], x[3]),      # q_BD
    (G,   x[2], x[3]),      # q_CD
    (G,   x[2], x[4]),      # q_CE
    ((2/3)*G, x[3], x[4]),  # q_DE
    (G,   x[4], p0)         # q_E0 
    ]

    q = [g * (i - j) for g, i, j in poisuilles_gildi] #[q_1A, q_AB, q_AC, q_BD, q_CD, q_CE, q_DE, q_E0]

    return q

In [ ]:
import numpy as np
from numpy import linalg as la
import math as m
from functions import poisuilles

# Fastar #
v = 1*10**(-3)
r = 5*10**(-2)
L = 100
G = (m.pi*(r**4))/(8*v*L)
p1 = 4.2*10**(6)
p0 = 0
Q_b = 7

# Skilgreining á fyljunum a og b fyrir ax = b
A = np.matrix('3 -1 -1 0 0; 1 -2 0 1 0; 1 0 -4 1 2; 0 3 3 -8 2; 0 0 6 2 -11')
B = np.matrix(f'{p1};{-Q_b/G}; 0; 0; 0')

# Nota linalg til að leysa fyrir x
x = la.solve(A,B)

# Reikna flæði í röri milli punkta með poisuilles jöfnunni
q = poisuilles(x,G,p1,p0)

# Print
print('q_1A         q_AB         q_AC         q_BD          q_CD        q_CE        q_DE         q_E0')
r = '  '.join(map(str, q)).replace("[[","").replace("]]","")
print(r)

In [ ]:
import functions as f
import numpy as np

import matplotlib.pyplot as plt

w = 2*np.pi/24 #Gögn úr dæmi

t = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23] #Gögn úr dæmi
b = [12.0, 14.0, 13.9, 12.3, 11.7, 8.9, 6.3, 6.5, 6.1, 5.8, 6.6, 9.3] #Gögn úr dæmi
a = np.matrix([[np.cos(w*x), np.sin(w*x), 1] for x in t]) #Fylkið A skv. aðverð minnstu kvarðata
aT = np.transpose(a) #Transpose af A
aTa = np.dot(aT,a) #Transpose af A margfaldað við A
aTaI = np.linalg.inv(aTa) #Adnhverfa af margefeldi Transpose A og A
x = np.dot(np.dot(aTaI, aT), b) #Andhverfan margfaldað við transpose A og b skv. setningu gefur x
print(x)

# plt.plot(t,b,'o')
# plt.plot(t,[f.qE0(i,x[0,0],x[0,1],x[0,2]) for i in t])
# plt.show()

In [ ]:
import numpy as np
import math as m
from functions import q_E0
from functions import poisuilles
from functions import bisection

# Gera posisuilles fallið að falli af p1
def p1_poisuilles(p1):

    G = (m.pi*((5*10**(-2))**4))/(8*(1*10**(-3))*100)
    x = np.array([
    2982653.95529531,
    2711991.61935227,
    2035970.24653365,
    2156123.62538856,
    1502551.70272537
    ])  
    p0 = 0
    q = poisuilles(x,G,p1,p0)
    return q[7]

# Skilgreina fasta
w = 2*np.pi/24
A = 1.32094476 
B = 4.01567095 
C = 9.45

# Búa til tíma vigur fyrir 100 tímaeiningar á einum sólahring
time = [24/100*i for i in range(1,101)]


for t in time:

    q = q_E0(A,B,C,w,t)

    


In [ ]:
import numpy as np
from numpy import linalg as LA

QB = 7.0
p0 = 0.0
p1 = 4.2e6
K  = 1.62e8   # your value
const_energy = (p0 - p1) / K   # ≈ -0.0259259...

def F(x):
    """
    x = [q1A, qAB, qAC, qBD, qCD, qCE, qDE, qE0]
    returns F(x) as an 8-vector
    """
    q1A, qAB, qAC, qBD, qCD, qCE, qDE, qE0 = x

    Fx = np.zeros(8)

    # 1) Node A
    Fx[0] = q1A - qAB - qAC

    # 2) Node B
    Fx[1] = qAB - qBD + QB

    # 3) Node C
    Fx[2] = qAC - qCD - qCE

    # 4) Node D
    Fx[3] = qBD + qCD - qDE

    # 5) Node E
    Fx[4] = qCE + qDE - qE0

    # 6) Loop 1: qAB|qAB| - qAC|qAC| + qBD|qBD| - qCD|qCD|
    Fx[5] = (
        qAB * abs(qAB)
        - qAC * abs(qAC)
        + qBD * abs(qBD)
        - qCD * abs(qCD)
    )

    # 7) Loop 2: qCD|qCD| - 0.5 qCE|qCE| + 1.5 qDE|qDE|
    Fx[6] = (
        qCD * abs(qCD)
        - 0.5 * qCE * abs(qCE)
        + 1.5 * qDE * abs(qDE)
    )

    # 8) Energy: (p0-p1)/K + q1A|q1A| + qAC|qAC| + 0.5 qCE|qCE| + qE0|qE0|
    Fx[7] = (
        const_energy
        + q1A * abs(q1A)
        + qAC * abs(qAC)
        + 0.5 * qCE * abs(qCE)
        + qE0 * abs(qE0)
    )

    return Fx


def DF(x):
    """
    Jacobi fylkið DF(x) for the above F(x)
    Uses d/dx (x|x|) = 2|x|
    """
    q1A, qAB, qAC, qBD, qCD, qCE, qDE, qE0 = x

    J = np.zeros((8, 8))

    # Row 1: F1 = q1A - qAB - qAC
    J[0, 0] =  1.0   # d/dq1A
    J[0, 1] = -1.0   # d/dqAB
    J[0, 2] = -1.0   # d/dqAC

    # Row 2: F2 = qAB - qBD + QB
    J[1, 1] =  1.0   # d/dqAB
    J[1, 3] = -1.0   # d/dqBD

    # Row 3: F3 = qAC - qCD - qCE
    J[2, 2] =  1.0   # d/dqAC
    J[2, 4] = -1.0   # d/dqCD
    J[2, 5] = -1.0   # d/dqCE

    # Row 4: F4 = qBD + qCD - qDE
    J[3, 3] =  1.0   # d/dqBD
    J[3, 4] =  1.0   # d/dqCD
    J[3, 6] = -1.0   # d/dqDE

    # Row 5: F5 = qCE + qDE - qE0
    J[4, 5] =  1.0   # d/dqCE
    J[4, 6] =  1.0   # d/dqDE
    J[4, 7] = -1.0   # d/dqE0

    # Row 6: F6 = qAB|qAB| - qAC|qAC| + qBD|qBD| - qCD|qCD|
    J[5, 1] =  2.0 * abs(qAB)   # d/dqAB
    J[5, 2] = -2.0 * abs(qAC)   # d/dqAC
    J[5, 3] =  2.0 * abs(qBD)   # d/dqBD
    J[5, 4] = -2.0 * abs(qCD)   # d/dqCD

    # Row 7: F7 = qCD|qCD| - 0.5 qCE|qCE| + 1.5 qDE|qDE|
    J[6, 4] =  2.0 * abs(qCD)        # d/dqCD
    J[6, 5] = -1.0 * abs(qCE)        # 2 * (-0.5)
    J[6, 6] =  3.0 * abs(qDE)        # 2 * (1.5)

    # Row 8: F8 = const + q1A|q1A| + qAC|qAC| + 0.5 qCE|qCE| + qE0|qE0|
    J[7, 0] = 2.0 * abs(q1A)     # d/dq1A
    J[7, 2] = 2.0 * abs(qAC)     # d/dqAC
    J[7, 5] = 1.0 * abs(qCE)     # 2 * 0.5
    J[7, 7] = 2.0 * abs(qE0)     # d/dqE0

    return J

def newtonmult(x0, tol):
    x = x0.copy()
    oldx = x + 2*tol
    while LA.norm(x - oldx, np.inf) > tol:
        oldx = x.copy()
        s = -LA.solve(DF(x), F(x))
        x = x + s
    return x

# Example usage:
x0 = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])  # or a better guess
tol = 1e-8
sol = newtonmult(x0, tol)
print("solution:", sol)
